In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob

fast_mode = (fls.env=='local')
clear_data = True

MainProcess
MainProcess


<module 'flg_support' from 'd:\\flagellar/code/core\\flg_support.py'>

In [2]:
# Prepare datasets
if not fast_mode:
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', fls.result_dir + '/many_full/', skip_download=clear_data)
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr/', skip_download=clear_data)
else:
    if clear_data:
        fls.remove_and_make_dir(fls.result_dir + '/many_full/')
        fls.remove_and_make_dir(fls.result_dir + '/many_abbr/')

In [ ]:
#fls.profiling=True
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count(start=0):
    print('Starting seed ', i)
    r = flg_runner.baseline_runner(fast_mode=fast_mode)
    r.git_commit_id = git_commit_id
    r.seed = i
    #r.base_model.train_data_selector.datasets = 'tom'
    #r.base_model.run_in_parallel = False
    base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]   
    if len(glob.glob(fls.result_dir + '/many_full/' + base_filename + '*'))>=1:
        print('Skipping')
        continue
    r.run()

    score_str = str(r.cv_score)[2:5]
    print(str(r.cv_score)[1:5])
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_' + score_str +' _f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_' + score_str +' _a.pickle'
    
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode:
        fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
        fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

Starting seed  0
1919 13
30 13
{'seed': 0, 'scale_percentile_value': 3.5478467492858172, 'img_size': 512, 'model_name': 'yolov8m', 'use_pretrained_weights': False, 'box_size': 17, 'trust': 0, 'fix_norm_bug': True, 'weight_decay': 0.0009127555772777217, 'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0, 'translate': 0.1, 'scale': 0.0, 'fliplr': 0.5, 'flipud': 0.0, 'degrees': 0, 'shear': 0.0, 'mosaic': 1.0, 'mixup': 0.2, 'erasing': 0.0, 'use_albumentations': False, 'confidence_threshold': 0.3556639342290926, 'include_multi_motor': False, 'tom': True, 'mba': True, 'aba': True, 'ycw': False}
.0
Starting seed  1
1919 13
30 13
{'seed': 1, 'scale_percentile_value': 3.047286498801027, 'img_size': 576, 'model_name': 'yolo11m', 'use_pretrained_weights': False, 'box_size': 29, 'trust': 5, 'fix_norm_bug': False, 'weight_decay': 0.00042332644897257566, 'hsv_h': 0.015, 'hsv_s': 0.0, 'hsv_v': 0.0, 'translate': 0.0, 'scale': 0.5274591760723646, 'fliplr': 0.5, 'flipud': 0.5, 'degrees': 0, 'shear': 0.0, 'mosaic'